In [46]:
import pandas as pd
import json
import folium
!pip install folium
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
cities = pd.read_csv("Map/city_all.csv")
cities.head(10)


In [ ]:
cities.rename(columns = {'state_abbr':'state','metric_name':'metric','city_name':'city'}, inplace = True)
cities


In [47]:
cities_new= cities[["state","metric","city"]]
cities_new

,state,metric,city
0,HI,Absenteeism,Honolulu
1,AL,Absenteeism,Birmingham
2,AL,Absenteeism,Hoover
3,AL,Absenteeism,Huntsville
4,AL,Absenteeism,Mobile
5,AL,Absenteeism,Montgomery
6,AL,Absenteeism,Tuscaloosa
7,AK,Absenteeism,Anchorage
8,AZ,Absenteeism,Avondale
9,AZ,Absenteeism,Chandler


In [48]:
# load GeoJSON
with open('Map/median_income.geojson', 'r') as jsonFile:
    data = json.load(jsonFile)
tmp = data

# remove ZIP codes not in our dataset
geozips = []
for i in range(len(tmp['features'])):
    if tmp['features'][i]['properties']['LOCALNAME'] in list(cities_new['city'].unique()):
        geozips.append(tmp['features'][i])

# creating new JSON object
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

# save JSON object as updated-file
open("updated-file.json", "w").write(
    json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': '))
)

14619840

In [49]:
import folium

# table = main table/data frame we read from (pandas DataFrame)
# zips = column name where ZIP codes are (string)
# mapped_feature = column name for feature we want to visualize (string)
# add_text = any additional commentary to be added in the map legend (string)

def create_map(table, zips, mapped_feature, add_text = ''):
    # reading of the updated GeoJSON file
    la_geo = r'updated-file.json'
    # initiating a Folium map with LA's longitude and latitude
    m = folium.Map(location = [34.0522, -118.2437], zoom_start = 11)
    # creating a choropleth map
    m.choropleth(
        geo_data = la_geo,
        fill_opacity = 0.7,
        line_opacity = 0.2,
        data = table,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.city',
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature],
        fill_color = 'RdYlGn',
        legend_name = (' ').join(mapped_feature.split('_')).title() + ' ' + add_text + ' Across LA'
    )
    folium.LayerControl().add_to(m)
    # save map with filename based on the feature of interest
    m.save(outfile = mapped_feature + '_map.html')

In [50]:
create_map(cities_new, 'state', 'metric', 'city')

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''